In [2]:
import requests
import json
import pandas as pd
import pymysql

In [18]:
# 영화배우, 감독정보???????????????????
cast_url="https://api.themoviedb.org/3/movie/238?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
response = requests.get(cast_url)
json_object = response.content
cast_data = json.loads(json_object)
cast_data
#print(cast_data_df)

{'adult': False,
 'backdrop_path': '/rSPw7tgCH9c6NqICZef4kZjFOQ5.jpg',
 'belongs_to_collection': {'id': 230,
  'name': 'The Godfather Collection',
  'poster_path': '/zqV8MGXfpLZiFVObLxpAI7wWonJ.jpg',
  'backdrop_path': '/pGl7NqNhyncju3Qu8LGA7owciMW.jpg'},
 'budget': 6000000,
 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}],
 'homepage': 'http://www.thegodfather.com/',
 'id': 238,
 'imdb_id': 'tt0068646',
 'original_language': 'en',
 'original_title': 'The Godfather',
 'overview': 'Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.',
 'popularity': 64.384,
 'poster_path': '/eEslKSwcqmiNS6va24Pbxf2UKmJ.jpg',
 'production_companies': [{'id': 4,
   'logo_path': '/fycMZt242LVjagMByZOLUGbCvv3.png',
   'name': 'Paramoun

In [13]:
#genre information crawling (for genre table)

genre_url="https://api.themoviedb.org/3/genre/movie/list?api_key=36927ad1d2817ff0fa31947e47d186fd&language=ko-KR"
response = requests.get(genre_url)
json_object = response.content
genre_data = json.loads(json_object)
genre_data
genre_data_df = pd.DataFrame(genre_data['genres'])
# print(genre_data_df)

#genre_data_df.to_csv("genre.csv")

{'genres': [{'id': 28, 'name': '액션'},
  {'id': 12, 'name': '모험'},
  {'id': 16, 'name': '애니메이션'},
  {'id': 35, 'name': '코미디'},
  {'id': 80, 'name': '범죄'},
  {'id': 99, 'name': '다큐멘터리'},
  {'id': 18, 'name': '드라마'},
  {'id': 10751, 'name': '가족'},
  {'id': 14, 'name': '판타지'},
  {'id': 36, 'name': '역사'},
  {'id': 27, 'name': '공포'},
  {'id': 10402, 'name': '음악'},
  {'id': 9648, 'name': '미스터리'},
  {'id': 10749, 'name': '로맨스'},
  {'id': 878, 'name': 'SF'},
  {'id': 10770, 'name': 'TV 영화'},
  {'id': 53, 'name': '스릴러'},
  {'id': 10752, 'name': '전쟁'},
  {'id': 37, 'name': '서부'}]}

In [19]:
# genre table에 영화 장르정보 삽입
#!pip install pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

cursor.execute('DELETE FROM genre')
for id, name in genre_data_df.values:
    # print("{0}/{1}".format(id, name))
    sql= "INSERT INTO genre VALUES(%s, %s)"
    cursor.execute(sql, (id, name))

conn.commit()
cursor.close()
conn.close()


In [20]:
# 2021년도 영화(2021-11-25일까지)크롤링 (for movie table)
movies_2021 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2021-01-01&vote_count.gte=100&primary_release_date.lte=2021-11-25&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2021.append(movies['results'])


movies = []
for row in movies_2021:
    for r in row:
       movies.append(r)
movies_2021_df=pd.DataFrame(movies)
movies_2021_modified = movies_2021_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2021_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

# movies_2021_modified.info()   #272 entries
# movies_2021_modified.to_csv("movies_2021.csv")

In [22]:
#2021년도 영화(2021-11-25일까지) movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

cursor.execute('DELETE FROM movie')

for row in movies_2021_modified.values:
     cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [23]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2021년도 movie_id, genre_id삽입

movie_genre_ids_2021_df=movies_2021_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()
cursor.execute('DELETE FROM movie_genre')

for movie_genre_ids in movie_genre_ids_2021_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        # print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [24]:
# 2020년도 영화 크롤링 (for movie table)
movies_2020 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2020-01-01&vote_count.gte=100&primary_release_date.lte=2020-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2020.append(movies['results'])


movies = []
for row in movies_2020:
    for r in row:
       movies.append(r)
movies_2020_df=pd.DataFrame(movies)
movies_2020_modified = movies_2020_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2020_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2020_modified.info()   #458 entries

In [25]:
movies_df_1=pd.concat([movies_2021_modified, movies_2020_modified], ignore_index=True)

In [27]:
#2020년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2020_modified.values:
     cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

IntegrityError: (1062, "Duplicate entry '593402' for key 'movie.PRIMARY'")

In [28]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2020년도 movie_id, genre_id삽입

movie_genre_ids_2020_df=movies_2020_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2020_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        # print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [29]:
# 2019년도 영화 크롤링 (for movie table)
movies_2019 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2019-01-01&vote_count.gte=100&primary_release_date.lte=2019-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2019.append(movies['results'])


movies = []
for row in movies_2019:
    for r in row:
       movies.append(r)
movies_2019_df=pd.DataFrame(movies)
movies_2019_modified = movies_2019_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2019_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2019_modified.info() # 458 entries
#movies_2019_modified.tail()


In [30]:
movie_df_2=pd.concat([movies_df_1, movies_2019_modified], ignore_index=True)

In [31]:
#2019년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2019_modified.values:
     cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [32]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2019년도 movie_id, genre_id삽입

movie_genre_ids_2019_df=movies_2019_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2019_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        # print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [33]:
# 2018년도 영화 크롤링 (for movie table)
movies_2018 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2018-01-01&vote_count.gte=100&primary_release_date.lte=2018-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2018.append(movies['results'])


movies = []
for row in movies_2018:
    for r in row:
       movies.append(r)
movies_2018_df=pd.DataFrame(movies)
movies_2018_modified = movies_2018_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2018_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2018_modified.info() # 707 entries
#movies_2018_modified.tail()

In [34]:
movie_df_3=pd.concat([movie_df_2, movies_2018_modified], ignore_index=True)

In [35]:
#2018년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2018_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [36]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2018년도 movie_id, genre_id삽입

movie_genre_ids_2018_df=movies_2018_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2018_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

IntegrityError: (1062, "Duplicate entry '519223-53' for key 'movie_genre.PRIMARY'")

In [37]:
# 2017년도 영화 크롤링 (for movie table)
movies_2017 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2017-01-01&vote_count.gte=100&primary_release_date.lte=2017-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2017.append(movies['results'])


movies = []
for row in movies_2017:
    for r in row:
       movies.append(r)
movies_2017_df=pd.DataFrame(movies)
movies_2017_modified = movies_2017_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2017_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2017_modified.info() # 719 entries
#movies_2017_modified.tail()

In [38]:
movie_df_4=pd.concat([movie_df_3, movies_2017_modified], ignore_index=True)

In [40]:
#2017년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2017_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [41]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2017년도 movie_id, genre_id삽입

movie_genre_ids_2017_df=movies_2017_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2017_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [42]:
# 2016년도 영화 크롤링 (for movie table)
movies_2016 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2016-01-01&vote_count.gte=100&primary_release_date.lte=2016-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2016.append(movies['results'])


movies = []
for row in movies_2016:
    for r in row:
       movies.append(r)
movies_2016_df=pd.DataFrame(movies)
movies_2016_modified = movies_2016_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2016_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2016_modified.info() # 673 entries
#movies_2016_modified.tail()

In [43]:
movie_df_5=pd.concat([movie_df_4, movies_2016_modified], ignore_index=True)

In [44]:
#2016년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2016_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [46]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2016년도 movie_id, genre_id삽입

movie_genre_ids_2016_df=movies_2016_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2016_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

IntegrityError: (1062, "Duplicate entry '379170-80' for key 'movie_genre.PRIMARY'")

In [47]:
# 2015년도 영화 크롤링 (for movie table)
movies_2015 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2015-01-01&vote_count.gte=100&primary_release_date.lte=2015-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2015.append(movies['results'])


movies = []
for row in movies_2015:
    for r in row:
       movies.append(r)
movies_2015_df=pd.DataFrame(movies)
movies_2015_modified = movies_2015_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2015_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

# movies_2015_modified.info() # 626 entries
# movies_2015_modified.tail()

In [48]:
movie_df_6=pd.concat([movie_df_5, movies_2015_modified], ignore_index=True)

In [49]:
#2015년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2015_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [50]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2015년도 movie_id, genre_id삽입

movie_genre_ids_2015_df=movies_2015_modified[['movie_id','genre_ids']]
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2015_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [51]:
# 2010~2014년도 영화 크롤링 (for movie table)
movies_2010_2014 = []
for page in range(1,250):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2010-01-01&vote_count.gte=100&primary_release_date.lte=2014-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2010_2014.append(movies['results'])


movies = []
for row in movies_2010_2014:
    for r in row:
       movies.append(r)
movies_2010_2014_df=pd.DataFrame(movies)
movies_2010_2014_modified = movies_2010_2014_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2010_2014_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#ovies_2010_2014_modified.info() # 2636 entries
#movies_2010_2014_modified.tail()

In [57]:
movie_df_7=pd.concat([movie_df_6, movies_2010_2014_modified], ignore_index=True)
movie_2010_2021= movie_df_7.to_csv("movies_2010_2021.csv")

In [53]:
#2010-2014년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()

for row in movies_2010_2014_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [54]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2010-2014년도 movie_id, genre_id삽입

movie_genre_ids_2010_2014_df=movies_2010_2014_modified[['movie_id','genre_ids']]
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2010_2014_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()